#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient(host=['mongodb://172.18.0.5:27017'])
#database = client['exercices']
#collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset=['launched'])
df_ks.rename(columns={'ID':'_id'}, inplace=True)

dataJson =df_ks.to_dict(orient='records')

In [6]:
dataJson


[{'_id': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2011-09-16',
  'goal': 2000.0,
  'launched': Timestamp('2011-08-17 06:31:31'),
  'pledged': 1145.0,
  'state': 'canceled',
  'backers': 24,
  'country': 'US',
  'usd pledged': 1145.0,
  'usd_pledged_real': 1145.0},
 {'_id': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  'launched': Timestamp('2012-07-23 20:46:48'),
  'pledged': 1851.0,
  'state': 'failed',
  'backers': 28,
  'country': 'US',
  'usd pledged': 1851.0,
  'usd_pledged_real': 1851.0},
 {'_id': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2016-07-19',
  'goal': 2000.0,
  'launched': Timestamp('2016-07-01 21:5

### Importer les données

In [7]:
db_exercices = client.exercices

In [8]:
collection_kickstarter = db_exercices['kickstarter']

In [9]:
collection_kickstarter.insert_many(dataJson)

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 872782264}, 'errmsg': 'E11000 duplicate key error collection: exercices.kickstarter index: _id_ dup key: { _id: 872782264 }', 'op': {'_id': 872782264, 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)', 'category': 'Rock', 'main_category': 'Music', 'currency': 'USD', 'deadline': '2011-09-16', 'goal': 2000.0, 'launched': Timestamp('2011-08-17 06:31:31'), 'pledged': 1145.0, 'state': 'canceled', 'backers': 24, 'country': 'US', 'usd pledged': 1145.0, 'usd_pledged_real': 1145.0}}], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}

In [10]:
db_exercices.list_collection_names()

['{query :{state:"successful"}, out:"sumSuccess"}', 'kickstarter']

## Question 1  

In [11]:
cur = collection_kickstarter.find().sort([("usd_pledged_real", -1)]).limit(5)
list(cur)

[{'_id': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': '50000.0',
  'launched': datetime.datetime(2014, 7, 8, 10, 14, 37),
  'pledged': '13285226.36',
  'state': 'successful',
  'backers': '62642',
  'country': 'US',
  'usd pledged': '13285226.36',
  'usd_pledged_real': 13285226.36},
 {'_id': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': '1000000.0',
  'launched': datetime.datetime(2016, 5, 24, 15, 49, 52),
  'pledged': '12779843.49',
  'state': 'successful',
  'backers': '66673',
  'country': 'US',
  'usd pledged': '12779843.49',
  'usd_pledged_real': 12779843.49},
 {'_id': 1033978702,
  'name': 'OUYA: A New Kind of Video Game Console',
  'category': 'Gaming Hardware',
  'main_category': 'Games'

## Question 2

In [20]:
cur = collection_kickstarter.count_documents({"state": "successful"})
print(cur)

53040


## Question 3

In [27]:
cur = collection_kickstarter.aggregate([{"$group" : {"_id" : "$category","projectNumberByCategory" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Animals', 'projectNumberByCategory': 97},
 {'_id': 'Textiles', 'projectNumberByCategory': 105},
 {'_id': 'Indie Rock', 'projectNumberByCategory': 2192},
 {'_id': 'Video', 'projectNumberByCategory': 147},
 {'_id': 'Radio & Podcasts', 'projectNumberByCategory': 349},
 {'_id': 'Candles', 'projectNumberByCategory': 168},
 {'_id': 'Romance', 'projectNumberByCategory': 74},
 {'_id': 'Architecture', 'projectNumberByCategory': 300},
 {'_id': 'Kids', 'projectNumberByCategory': 109},
 {'_id': 'Gaming Hardware', 'projectNumberByCategory': 178},
 {'_id': 'Performances', 'projectNumberByCategory': 414},
 {'_id': 'Photobooks', 'projectNumberByCategory': 608},
 {'_id': 'Poetry', 'projectNumberByCategory': 532},
 {'_id': 'Crochet', 'projectNumberByCategory': 61},
 {'_id': 'Latin', 'projectNumberByCategory': 63},
 {'_id': 'Mixed Media', 'projectNumberByCategory': 1118},
 {'_id': "Farmer's Markets", 'projectNumberByCategory': 175},
 {'_id': 'Punk', 'projectNumberByCategory': 127},
 {'_id': 'Fo

## Question 4

In [53]:
import datetime
date = datetime.datetime(2016,1,1)
cur = collection_kickstarter.count_documents({"$and":[{"launched":{"$lt": date}}, {"country":"FR"}]})
print(cur)

330


## Question 5

In [35]:
cur = collection_kickstarter.find({"$and":[{"goal":{"$gt": 200000}}, {"country":"US"}]})
list(cur)

[{'_id': 866634482,
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2012-09-14',
  'goal': 287000.0,
  'launched': datetime.datetime(2012, 8, 13, 23, 14, 2),
  'pledged': 1465.0,
  'state': 'failed',
  'backers': 11,
  'country': 'US',
  'usd pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': 993194166,
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2012-06-01',
  'goal': 250000.0,
  'launched': datetime.datetime(2012, 5, 1, 20, 49, 58),
  'pledged': 23680.54,
  'state': 'failed',
  'backers': 409,
  'country': 'US',
  'usd pledged': 23680.54,
  'usd_pledged_real': 23680.54},
 {'_id': 1147175344,
  'name': 'Shine On New World',
  'category': 'Theater',
  'main_category': 'Theater',
  'currency': 'USD',
  'deadline': '2013-11-04',
  'goal': 300000.0,
  'launched': datetime.datetime(2013, 9, 30, 18

## Question 6 

In [48]:
cur = collection_kickstarter.count_documents({"name" :{'$regex':'.*Sport.*'}})
print(cur)

323
